# First Model: CNN

## Import packages

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import itertools
import os
import glob 
import seaborn as sns
import tensorflow as tf
import multiprocessing as mp
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Lambda
from tensorflow.keras.models import Model
from astropy.stats import sigma_clip
from tqdm import tqdm
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor, as_completed

sns.set_theme(style='dark')
palette = sns.color_palette('muted')
pd.set_option('display.max_columns', None)

print('TF version:', tf.__version__)

2025-09-28 05:14:38.350336: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759036478.690069      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759036478.784967      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TF version: 2.18.0


## Load and calibrate the data

In [2]:
path_folder = '/kaggle/input/ariel-data-challenge-2025/' 
path_out = '/kaggle/tmp/data_light_raw/'

if not os.path.exists(path_out):
    os.makedirs(path_out)
    print(f"Directory {path_out} created.")
else:
    print(f"Directory {path_out} already exists.")

CHUNKS_SIZE = 4

Directory /kaggle/tmp/data_light_raw/ created.


In [3]:
def ADC_convert(signal, gain=0.4369, offset=-1000):
    """The Analog-to-Digital Conversion (adc) is performed by the detector to convert
    the pixel voltage into an integer number. Since we are using the same conversion number 
    this year, we have simply hard-coded it inside."""
    signal = signal.astype(np.float64)
    signal /= gain
    signal += offset
    return signal

def mask_hot_dead(signal, dead, dark):
    hot = sigma_clip(
        dark, sigma=5, maxiters=5
    ).mask
    hot = np.tile(hot, (signal.shape[0], 1, 1))
    dead = np.tile(dead, (signal.shape[0], 1, 1))
    signal = np.ma.masked_where(dead, signal)
    signal = np.ma.masked_where(hot, signal)
    return signal

def apply_linear_corr(linear_corr, clean_signal):
    linear_corr = np.flip(linear_corr, axis=0)
    for x, y in itertools.product(
                range(clean_signal.shape[1]), range(clean_signal.shape[2])
            ):
        poli = np.poly1d(linear_corr[:, x, y])
        clean_signal[:, x, y] = poli(clean_signal[:, x, y])
    return clean_signal

def clean_dark(signal, dead, dark, dt):
    dark = np.ma.masked_where(dead, dark)
    dark = np.tile(dark, (signal.shape[0], 1, 1))
    signal -= dark * dt[:, np.newaxis, np.newaxis]
    return signal

def get_cds(signal):
    cds = signal[:,1::2,:,:] - signal[:,::2,:,:]
    return cds

def bin_obs(cds_signal, binning):
    cds_transposed = cds_signal.transpose(0,1,3,2)
    cds_binned = np.zeros((cds_transposed.shape[0], cds_transposed.shape[1] // binning, cds_transposed.shape[2], cds_transposed.shape[3]))
    for i in range(cds_transposed.shape[1] // binning):
        cds_binned[:,i,:,:] = np.sum(cds_transposed[:,i*binning:(i+1)*binning,:,:], axis=1)
    return cds_binned

def correct_flat_field(flat, dead, signal):
    flat = flat.transpose(1, 0)
    dead = dead.transpose(1, 0)
    flat = np.ma.masked_where(dead, flat)
    flat = np.tile(flat, (signal.shape[0], 1, 1))
    signal /= flat
    return signal

def get_index(files, CHUNKS_SIZE):
    index = []
    for file in files:
        file_name = file.split('/')[-1]
        if file_name.split('_')[0] == 'AIRS-CH0' and file_name.split('_')[1] == 'signal' and file_name.split('_')[2] == '0.parquet':
            file_index = os.path.basename(os.path.dirname(file))
            index.append(int(file_index))
    index = np.array(index)
    index = np.sort(index) 
    # credit to DennisSakva
    index = np.array_split(index, len(index) // CHUNKS_SIZE)
    
    return index

In [4]:
files = glob.glob(os.path.join(path_folder + 'train/', '*/*'))

In [5]:
index = get_index(files, CHUNKS_SIZE)
print(len(index[0]))

4


In [6]:
axis_info = pd.read_parquet(os.path.join(path_folder,'axis_info.parquet'))
DO_MASK = False
DO_THE_NL_CORR = False
DO_DARK = False
DO_FLAT = False
TIME_BINNING = True

cut_inf, cut_sup = 39, 321
l = cut_sup - cut_inf

In [7]:
def load_calibration_data_batch(path_folder, index_chunk, cut_inf, cut_sup, dataset):
    """Load all calibration data for the chunk at once"""
    calibration_data = {}
    
    for idx in index_chunk:
        calibration_data[idx] = {}
        
        # AIRS calibration data
        airs_flat = pd.read_parquet(os.path.join(path_folder, f'{dataset}/{idx}/AIRS-CH0_calibration_0/flat.parquet'))
        calibration_data[idx]['airs_flat'] = airs_flat.values.astype(np.float32).reshape((32, 356))[:, cut_inf:cut_sup]
        
        airs_dark = pd.read_parquet(os.path.join(path_folder, f'{dataset}/{idx}/AIRS-CH0_calibration_0/dark.parquet'))
        calibration_data[idx]['airs_dark'] = airs_dark.values.astype(np.float32).reshape((32, 356))[:, cut_inf:cut_sup]
        
        airs_dead = pd.read_parquet(os.path.join(path_folder, f'{dataset}/{idx}/AIRS-CH0_calibration_0/dead.parquet'))
        calibration_data[idx]['airs_dead'] = airs_dead.values.astype(np.float32).reshape((32, 356))[:, cut_inf:cut_sup]
        
        airs_linear = pd.read_parquet(os.path.join(path_folder, f'{dataset}/{idx}/AIRS-CH0_calibration_0/linear_corr.parquet'))
        calibration_data[idx]['airs_linear'] = airs_linear.values.astype(np.float32).reshape((6, 32, 356))[:, :, cut_inf:cut_sup]
        
        # FGS1 calibration data
        fgs_flat = pd.read_parquet(os.path.join(path_folder, f'{dataset}/{idx}/FGS1_calibration_0/flat.parquet'))
        calibration_data[idx]['fgs_flat'] = fgs_flat.values.astype(np.float32).reshape((32, 32))
        
        fgs_dark = pd.read_parquet(os.path.join(path_folder, f'{dataset}/{idx}/FGS1_calibration_0/dark.parquet'))
        calibration_data[idx]['fgs_dark'] = fgs_dark.values.astype(np.float32).reshape((32, 32))
        
        fgs_dead = pd.read_parquet(os.path.join(path_folder, f'{dataset}/{idx}/FGS1_calibration_0/dead.parquet'))
        calibration_data[idx]['fgs_dead'] = fgs_dead.values.astype(np.float32).reshape((32, 32))
        
        fgs_linear = pd.read_parquet(os.path.join(path_folder, f'{dataset}/{idx}/FGS1_calibration_0/linear_corr.parquet'))
        calibration_data[idx]['fgs_linear'] = fgs_linear.values.astype(np.float32).reshape((6, 32, 32))
    
    return calibration_data

In [8]:
def process_single_observation(args):
    """Process a single observation with all AIRS and FGS1 cleaning steps"""
    # Unpack the arguments
    (i, index_chunk, path_folder, cut_inf, cut_sup, l, axis_info, calibration_data, DO_MASK, DO_THE_NL_CORR, DO_DARK) = args
    
    idx = index_chunk[i]
    
    # AIRS Processing
    # Load AIRS signal data
    df = pd.read_parquet(os.path.join(path_folder, f'train/{idx}/AIRS-CH0_signal_0.parquet'))
    signal = df.values.astype(np.float32).reshape((df.shape[0], 32, 356))

    # 1. ADC Conversion
    signal = ADC_convert(signal)
    dt_airs = axis_info['AIRS-CH0-integration_time'].dropna().values
    dt_airs[1::2] += 0.1
    chopped_signal = signal[:, :, cut_inf:cut_sup]
    del signal, df
    
    # Get pre-loaded calibration data for AIRS
    flat = calibration_data[idx]['airs_flat']
    dark = calibration_data[idx]['airs_dark'] 
    dead_airs = calibration_data[idx]['airs_dead']
    linear_corr = calibration_data[idx]['airs_linear']

    # 2. Mask Hot/Dead Pixels
    if DO_MASK:
        chopped_signal = mask_hot_dead(chopped_signal, dead_airs, dark)

    # 3. Linearity Correction
    if DO_THE_NL_CORR: 
        linear_corr_signal = apply_linear_corr(linear_corr, chopped_signal)
        chopped_signal = linear_corr_signal

    # 4. Dark Current Subtraction
    if DO_DARK: 
        cleaned_signal = clean_dark(chopped_signal, dead_airs, dark, dt_airs)
        chopped_signal = cleaned_signal

    # Store AIRS result
    airs_result = chopped_signal
    
    # FGS1 Processing
    # Load FGS1 signal data
    df = pd.read_parquet(os.path.join(path_folder, f'train/{idx}/FGS1_signal_0.parquet'))
    fgs_signal = df.values.astype(np.float32).reshape((df.shape[0], 32, 32))

    # 1. ADC Conversion
    fgs_signal = ADC_convert(fgs_signal)
    dt_fgs1 = np.ones(len(fgs_signal)) * 0.1
    dt_fgs1[1::2] += 0.1
    chopped_FGS1 = fgs_signal
    del fgs_signal, df
    
    # Get pre-loaded calibration data for FGS1
    flat = calibration_data[idx]['fgs_flat']
    dark = calibration_data[idx]['fgs_dark']
    dead_fgs1 = calibration_data[idx]['fgs_dead']
    linear_corr = calibration_data[idx]['fgs_linear']

    # 2. Mask Hot/Dead pixels
    if DO_MASK:
        chopped_FGS1 = mask_hot_dead(chopped_FGS1, dead_fgs1, dark)

    # 3. Linearity Correction
    if DO_THE_NL_CORR: 
        linear_corr_signal = apply_linear_corr(linear_corr, chopped_FGS1)
        chopped_FGS1 = linear_corr_signal

    # 4. Dark Current Subtraction
    if DO_DARK: 
        cleaned_signal = clean_dark(chopped_FGS1, dead_fgs1, dark, dt_fgs1)
        chopped_FGS1 = cleaned_signal

    # Store FGS1 result
    fgs_result = chopped_FGS1
    
    # Return the processed results
    return i, airs_result, fgs_result

In [9]:
def process_single_observation_test(args):
    """Process a single observation with all AIRS and FGS1 cleaning steps"""
    # Unpack the arguments
    (i, index_chunk, path_folder, cut_inf, cut_sup, l, axis_info, calibration_data, DO_MASK, DO_THE_NL_CORR, DO_DARK) = args
    
    idx = index_chunk[i]
    
    # AIRS Processing
    # Load AIRS signal data
    df = pd.read_parquet(os.path.join(path_folder, f'test/{idx}/AIRS-CH0_signal_0.parquet'))
    signal = df.values.astype(np.float32).reshape((df.shape[0], 32, 356))

    # 1. ADC Conversion
    signal = ADC_convert(signal)
    dt_airs = axis_info['AIRS-CH0-integration_time'].dropna().values
    dt_airs[1::2] += 0.1
    chopped_signal = signal[:, :, cut_inf:cut_sup]
    del signal, df
    
    # Get pre-loaded calibration data for AIRS
    flat = calibration_data[idx]['airs_flat']
    dark = calibration_data[idx]['airs_dark'] 
    dead_airs = calibration_data[idx]['airs_dead']
    linear_corr = calibration_data[idx]['airs_linear']

    # 2. Mask Hot/Dead Pixels
    if DO_MASK:
        chopped_signal = mask_hot_dead(chopped_signal, dead_airs, dark)

    # 3. Linearity Correction
    if DO_THE_NL_CORR: 
        linear_corr_signal = apply_linear_corr(linear_corr, chopped_signal)
        chopped_signal = linear_corr_signal

    # 4. Dark Current Subtraction
    if DO_DARK: 
        cleaned_signal = clean_dark(chopped_signal, dead_airs, dark, dt_airs)
        chopped_signal = cleaned_signal

    # Store AIRS result
    airs_result = chopped_signal
    
    # FGS1 Processing
    # Load FGS1 signal data
    df = pd.read_parquet(os.path.join(path_folder, f'test/{idx}/FGS1_signal_0.parquet'))
    fgs_signal = df.values.astype(np.float32).reshape((df.shape[0], 32, 32))

    # 1. ADC Conversion
    fgs_signal = ADC_convert(fgs_signal)
    dt_fgs1 = np.ones(len(fgs_signal)) * 0.1
    dt_fgs1[1::2] += 0.1
    chopped_FGS1 = fgs_signal
    del fgs_signal, df
    
    # Get pre-loaded calibration data for FGS1
    flat = calibration_data[idx]['fgs_flat']
    dark = calibration_data[idx]['fgs_dark']
    dead_fgs1 = calibration_data[idx]['fgs_dead']
    linear_corr = calibration_data[idx]['fgs_linear']

    # 2. Mask Hot/Dead pixels
    if DO_MASK:
        chopped_FGS1 = mask_hot_dead(chopped_FGS1, dead_fgs1, dark)

    # 3. Linearity Correction
    if DO_THE_NL_CORR: 
        linear_corr_signal = apply_linear_corr(linear_corr, chopped_FGS1)
        chopped_FGS1 = linear_corr_signal

    # 4. Dark Current Subtraction
    if DO_DARK: 
        cleaned_signal = clean_dark(chopped_FGS1, dead_fgs1, dark, dt_fgs1)
        chopped_FGS1 = cleaned_signal

    # Store FGS1 result
    fgs_result = chopped_FGS1
    
    # Return the processed results
    return i, airs_result, fgs_result

In [10]:
for n, index_chunk in enumerate(tqdm(index)):
    # Load all calibration data once at the beginning 
    calibration_data = load_calibration_data_batch(path_folder, index_chunk, cut_inf, cut_sup, 'train')
    
    # Pre-allocate output arrays
    AIRS_CH0_clean = np.zeros((CHUNKS_SIZE, 11250, 32, l), dtype=np.float32)
    FGS1_clean = np.zeros((CHUNKS_SIZE, 135000, 32, 32), dtype=np.float32)
    
    # Parallel Processing
    # Determine number of workers 
    num_workers = min(2, CHUNKS_SIZE)
    
    # Prepare arguments for each observation
    args_list = []
    for i in range(CHUNKS_SIZE):
        args = (i, index_chunk, path_folder, cut_inf, cut_sup, l, axis_info, calibration_data, DO_MASK, DO_THE_NL_CORR, DO_DARK)
        args_list.append(args)
        
    # Process observations in parallel
    results = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:        
        # Submit all tasks to the thread pool
        future_to_index = {executor.submit(process_single_observation, args): args[0] for args in args_list}
                
        # Collect results as they complete
        for future in tqdm(as_completed(future_to_index), total=CHUNKS_SIZE, desc=f"Processing observations"):
            i, airs_result, fgs_result = future.result()
            results.append((i, airs_result, fgs_result))
        
    # Sort results by observation index (i) to maintain order
    results.sort(key=lambda x: x[0])
    
    # Store results in your existing arrays
    for result in results:
        i, airs_result, fgs_result = result
        AIRS_CH0_clean[i] = airs_result
        FGS1_clean[i] = fgs_result
        
    # 5. Get Correlated Double Sampling
    AIRS_cds = get_cds(AIRS_CH0_clean)
    FGS1_cds = get_cds(FGS1_clean)
    
    del AIRS_CH0_clean, FGS1_clean
    
    # 6. (Optional) Time Binning (to reduce space)
    if TIME_BINNING:
        AIRS_cds_binned = bin_obs(AIRS_cds, binning=30)
        FGS1_cds_binned = bin_obs(FGS1_cds, binning=30*12)
    else:
        AIRS_cds = AIRS_cds.transpose(0,1,3,2)
        AIRS_cds_binned = AIRS_cds
        FGS1_cds = FGS1_cds.transpose(0,1,3,2)
        FGS1_cds_binned = FGS1_cds
    
    del AIRS_cds, FGS1_cds

    # 7. Flat Field Correction - use pre-loaded calibration data
    for i in range(CHUNKS_SIZE):
        if DO_FLAT:
            flat_airs = calibration_data[index_chunk[i]]['airs_flat']  # Fixed: added [i]
            flat_fgs = calibration_data[index_chunk[i]]['fgs_flat']    # Fixed: added [i]
            dead_airs = calibration_data[index_chunk[i]]['airs_dead']  # Fixed: added [i]
            dead_fgs1 = calibration_data[index_chunk[i]]['fgs_dead']   # Fixed: added [i]
            
            corrected_AIRS_cds_binned = correct_flat_field(flat_airs, dead_airs, AIRS_cds_binned[i])
            AIRS_cds_binned[i] = corrected_AIRS_cds_binned
            corrected_FGS1_cds_binned = correct_flat_field(flat_fgs, dead_fgs1, FGS1_cds_binned[i])
            FGS1_cds_binned[i] = corrected_FGS1_cds_binned

    # Save data
    np.save(os.path.join(path_out, 'AIRS_clean_train_{}.npy'.format(n)), AIRS_cds_binned)
    np.save(os.path.join(path_out, 'FGS1_train_{}.npy'.format(n)), FGS1_cds_binned)
    del AIRS_cds_binned, FGS1_cds_binned, calibration_data

100%|██████████| 275/275 [1:29:43<00:00, 19.58s/it]


In [11]:
TRAIN_CHUNKS_SIZE = CHUNKS_SIZE
CHUNKS_SIZE = 1
test_files = glob.glob(os.path.join(path_folder + 'test/', '*/*'))
test_index = get_index(test_files, CHUNKS_SIZE)

for n, index_chunk in enumerate(tqdm(test_index)):
    # Load all calibration data once at the beginning 
    calibration_data = load_calibration_data_batch(path_folder, index_chunk, cut_inf, cut_sup, 'test')
    
    # Pre-allocate output arrays
    AIRS_CH0_clean = np.zeros((CHUNKS_SIZE, 11250, 32, l), dtype=np.float32)
    FGS1_clean = np.zeros((CHUNKS_SIZE, 135000, 32, 32), dtype=np.float32)
    
    # Parallel Processing
    # Determine number of workers (start with 2 to be safe)
    num_workers = min(2, CHUNKS_SIZE)
    
    # Prepare arguments for each observation
    args_list = []
    for i in range(CHUNKS_SIZE):
        args = (i, index_chunk, path_folder, cut_inf, cut_sup, l, axis_info, calibration_data, DO_MASK, DO_THE_NL_CORR, DO_DARK)
        args_list.append(args)
        
    # Process observations in parallel
    results = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:        
        # Submit all tasks to the thread pool
        future_to_index = {executor.submit(process_single_observation_test, args): args[0] for args in args_list}
                
        # Collect results as they complete
        for future in tqdm(as_completed(future_to_index), total=CHUNKS_SIZE, desc=f"Processing observations"):
            i, airs_result, fgs_result = future.result()
            results.append((i, airs_result, fgs_result))
        
    # Sort results by observation index (i) to maintain order
    results.sort(key=lambda x: x[0])
    
    # Store results in your existing arrays
    for result in results:
        i, airs_result, fgs_result = result
        AIRS_CH0_clean[i] = airs_result
        FGS1_clean[i] = fgs_result
        
    # 5. Get Correlated Double Sampling
    AIRS_cds = get_cds(AIRS_CH0_clean)
    FGS1_cds = get_cds(FGS1_clean)
    
    del AIRS_CH0_clean, FGS1_clean
    
    # 6. (Optional) Time Binning (to reduce space)
    if TIME_BINNING:
        AIRS_cds_binned = bin_obs(AIRS_cds, binning=30)
        FGS1_cds_binned = bin_obs(FGS1_cds, binning=30*12)
    else:
        AIRS_cds = AIRS_cds.transpose(0,1,3,2)
        AIRS_cds_binned = AIRS_cds
        FGS1_cds = FGS1_cds.transpose(0,1,3,2)
        FGS1_cds_binned = FGS1_cds
    
    del AIRS_cds, FGS1_cds

    # 7. Flat Field Correction - use pre-loaded calibration data
    for i in range(CHUNKS_SIZE):
        if DO_FLAT:
            flat_airs = calibration_data[index_chunk[i]]['airs_flat']  # Fixed: added [i]
            flat_fgs = calibration_data[index_chunk[i]]['fgs_flat']    # Fixed: added [i]
            dead_airs = calibration_data[index_chunk[i]]['airs_dead']  # Fixed: added [i]
            dead_fgs1 = calibration_data[index_chunk[i]]['fgs_dead']   # Fixed: added [i]
            
            corrected_AIRS_cds_binned = correct_flat_field(flat_airs, dead_airs, AIRS_cds_binned[i])
            AIRS_cds_binned[i] = corrected_AIRS_cds_binned
            corrected_FGS1_cds_binned = correct_flat_field(flat_fgs, dead_fgs1, FGS1_cds_binned[i])
            FGS1_cds_binned[i] = corrected_FGS1_cds_binned

    # Save data
    np.save(os.path.join(path_out, 'AIRS_clean_test_{}.npy'.format(n)), AIRS_cds_binned)
    np.save(os.path.join(path_out, 'FGS1_test_{}.npy'.format(n)), FGS1_cds_binned)
    del AIRS_cds_binned, FGS1_cds_binned, calibration_data

100%|██████████| 1/1 [00:07<00:00,  7.33s/it]


In [12]:
def load_data(file, chunk_size, nb_files): 
    data0 = np.load(file + '_0.npy')
    data_all = np.zeros((nb_files * chunk_size, data0.shape[1], data0.shape[2], data0.shape[3]))
    data_all[:chunk_size] = data0
    for i in range(1, nb_files): 
        data_all[i * chunk_size : (i+1) * chunk_size] = np.load(file + '_{}.npy'.format(i))
    return data_all 

data_train_AIRS = load_data(path_out + 'AIRS_clean_train', TRAIN_CHUNKS_SIZE, len(index)) 
data_train_FGS = load_data(path_out + 'FGS1_train', TRAIN_CHUNKS_SIZE, len(index))

print(data_train_AIRS.shape)
print(data_train_FGS.shape)

(1100, 187, 282, 32)
(1100, 187, 32, 32)


In [13]:
data_test_AIRS = load_data(path_out + 'AIRS_clean_test', CHUNKS_SIZE, 1) 
data_test_FGS = load_data(path_out + 'FGS1_test', CHUNKS_SIZE, 1)

print(data_test_AIRS.shape)
print(data_test_FGS.shape)

(1, 187, 282, 32)
(1, 187, 32, 32)


In [14]:
df_train = pd.read_csv(path_folder + 'train.csv')
df_train.set_index('planet_id', inplace=True)

df_train = df_train[df_train.index.isin(index[0])]

print(df_train.shape)
df_train.head()

(4, 283)


,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,wl_10,wl_11,wl_12,wl_13,wl_14,wl_15,wl_16,wl_17,wl_18,wl_19,wl_20,wl_21,wl_22,wl_23,wl_24,wl_25,wl_26,wl_27,wl_28,wl_29,wl_30,wl_31,wl_32,wl_33,wl_34,wl_35,wl_36,wl_37,wl_38,wl_39,wl_40,wl_41,wl_42,wl_43,wl_44,wl_45,wl_46,wl_47,wl_48,wl_49,wl_50,wl_51,wl_52,wl_53,wl_54,wl_55,wl_56,wl_57,wl_58,wl_59,wl_60,wl_61,wl_62,wl_63,wl_64,wl_65,wl_66,wl_67,wl_68,wl_69,wl_70,wl_71,wl_72,wl_73,wl_74,wl_75,wl_76,wl_77,wl_78,wl_79,wl_80,wl_81,wl_82,wl_83,wl_84,wl_85,wl_86,wl_87,wl_88,wl_89,wl_90,wl_91,wl_92,wl_93,wl_94,wl_95,wl_96,wl_97,wl_98,wl_99,wl_100,wl_101,wl_102,wl_103,wl_104,wl_105,wl_106,wl_107,wl_108,wl_109,wl_110,wl_111,wl_112,wl_113,wl_114,wl_115,wl_116,wl_117,wl_118,wl_119,wl_120,wl_121,wl_122,wl_123,wl_124,wl_125,wl_126,wl_127,wl_128,wl_129,wl_130,wl_131,wl_132,wl_133,wl_134,wl_135,wl_136,wl_137,wl_138,wl_139,wl_140,wl_141,wl_142,wl_143,wl_144,wl_145,wl_146,wl_147,wl_148,wl_149,wl_150,wl_151,wl_152,wl_153,wl_154,wl_155,wl_156,wl_157,wl_158,wl_159,wl_160,wl_161,wl_162,wl_163,wl_164,wl_165,wl_166,wl_167,wl_168,wl_169,wl_170,wl_171,wl_172,wl_173,wl_174,wl_175,wl_176,wl_177,wl_178,wl_179,wl_180,wl_181,wl_182,wl_183,wl_184,wl_185,wl_186,wl_187,wl_188,wl_189,wl_190,wl_191,wl_192,wl_193,wl_194,wl_195,wl_196,wl_197,wl_198,wl_199,wl_200,wl_201,wl_202,wl_203,wl_204,wl_205,wl_206,wl_207,wl_208,wl_209,wl_210,wl_211,wl_212,wl_213,wl_214,wl_215,wl_216,wl_217,wl_218,wl_219,wl_220,wl_221,wl_222,wl_223,wl_224,wl_225,wl_226,wl_227,wl_228,wl_229,wl_230,wl_231,wl_232,wl_233,wl_234,wl_235,wl_236,wl_237,wl_238,wl_239,wl_240,wl_241,wl_242,wl_243,wl_244,wl_245,wl_246,wl_247,wl_248,wl_249,wl_250,wl_251,wl_252,wl_253,wl_254,wl_255,wl_256,wl_257,wl_258,wl_259,wl_260,wl_261,wl_262,wl_263,wl_264,wl_265,wl_266,wl_267,wl_268,wl_269,wl_270,wl_271,wl_272,wl_273,wl_274,wl_275,wl_276,wl_277,wl_278,wl_279,wl_280,wl_281,wl_282,wl_283
planet_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
34983,0.018291,0.018088,0.018087,0.018085,0.018084,0.018084,0.018084,0.018084,0.018085,0.018084,0.018083,0.018080,0.018076,0.018072,0.018068,0.018063,0.018060,0.018060,0.018062,0.018069,0.018079,0.018091,0.018103,0.018111,0.018115,0.018117,0.018119,0.018123,0.018129,0.018136,0.018139,0.018137,0.018133,0.018131,0.018131,0.018132,0.018133,0.018134,0.018135,0.018136,0.018137,0.018136,0.018134,0.018133,0.018132,0.018132,0.018131,0.018129,0.018127,0.018126,0.018126,0.018126,0.018124,0.018119,0.018115,0.018111,0.018108,0.018106,0.018104,0.018102,0.018100,0.018098,0.018096,0.018095,0.018095,0.018095,0.018095,0.018094,0.018092,0.018090,0.018089,0.018088,0.018087,0.018084,0.018082,0.018080,0.018079,0.018079,0.018080,0.018080,0.018079,0.018076,0.018074,0.018073,0.018076,0.018082,0.018093,0.018105,0.018116,0.018124,0.018130,0.018134,0.018136,0.018136,0.018136,0.018137,0.018137,0.018137,0.018138,0.018140,0.018144,0.018147,0.018148,0.018148,0.018150,0.018152,0.018154,0.018156,0.018157,0.018157,0.018158,0.018158,0.018158,0.018157,0.018158,0.018159,0.018160,0.018161,0.018162,0.018163,0.018164,0.018167,0.018169,0.018171,0.018171,0.018171,0.018170,0.018170,0.018169,0.018168,0.018167,0.018167,0.018169,0.018173,0.018174,0.018171,0.018167,0.018165,0.018164,0.018164,0.018164,0.018163,0.018162,0.018162,0.018161,0.018160,0.018159,0.018160,0.018161,0.018163,0.018165,0.018166,0.018168,0.018171,0.018173,0.018174,0.018176,0.018178,0.018178,0.018175,0.018173,0.018171,0.018170,0.018169,0.018168,0.018166,0.018165,0.018165,0.018163,0.018159,0.018155,0.018154,0.018152,0.018148,0.018144,0.018142,0.018142,0.018146,0.018155,0.018172,0.018192,0.018207,0.018211,0.018208,0.018200,0.018190,0.018179,0.018169,0.018161,0.018156,0.018152,0.018150,0.018147,0.018147,0.018150,0.018154,0.018156,0.018157,0.018157,0.018157,0.018156,0.018155,0.018155,

## Split into train and validation sets

In [15]:
n = round(.8 * len(df_train))
n

3

In [16]:
train_AIRS = data_train_AIRS[:n]
val_AIRS = data_train_AIRS[n:]
print(len(train_AIRS), len(val_AIRS))

train_FGS = data_train_FGS[:n]
val_FGS = data_train_FGS[n:]
print(len(train_FGS), len(val_FGS))

train_labels = df_train.iloc[:n,:] 
val_labels = df_train.iloc[n:,:]
print(train_labels.shape, val_labels.shape)

3 1097
3 1097
(3, 283) (1, 283)


## Define the model

In [17]:
print(train_AIRS.shape)
print(train_labels.shape)

(3, 187, 282, 32)
(3, 283)


In [18]:
inputs = Input(shape=(187, 282, 32), name='inputs')

x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = Flatten()(x)

x = Dense(64, activation='relu')(x)

# Two output heads
mean_output = Dense(283, activation='linear', name='mean')(x)  
log_std_output = Dense(283, activation='linear', name='log_std')(x) 
std_output = Lambda(lambda x: tf.exp(0.5 * x), name='std')(log_std_output)

# Concatenate outputs for submission
outputs = Concatenate(name='outputs')([mean_output, std_output])

model = Model(inputs=inputs, outputs=outputs)
model.summary()

I0000 00:00:1759041981.125864      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1759041981.126591      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer) │ (None, 187, 282,  │          0 │ -                 │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 185, 280,  │      9,248 │ inputs[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 92, 140,   │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 90, 138,   │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 45, 69,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 43, 67,    │     36,928 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 184384)    │          0 │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │ 11,800,640 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ log_std (Dense)     │ (None, 283)       │     18,395 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mean (Dense)        │ (None, 283)       │     18,395 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ std (Lambda)        │ (None, 283)       │          0 │ log_std[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ outputs             │ (None, 566)       │          0 │ mean[0][0],       │
│ (Concatenate)       │                   │            │ std[0][0]         │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,902,102 (45.40 MB)

 Trainable params: 11,902,102 (45.40 MB)

 Non-trainable params: 0 (0.00 B)

## Compile and train the model

In [19]:
def nll_loss(y_true, y_pred):
    mu, std = y_pred[:, :283], y_pred[:, 283:]
    return tf.reduce_mean(0.5 * tf.math.log(2 * np.pi * std**2) + 0.5 * ((y_true - mu)**2 / std**2))

model.compile(optimizer = 'adam', loss = nll_loss)

## Generate predictions

In [20]:
# Generate predictions
predictions = model.predict(data_test_AIRS)  
means = predictions[:, :283]  
stds = predictions[:, 283:] 

# Create submission DataFrame
df_sample = pd.read_csv(path_folder + 'sample_submission.csv')
df_submission = pd.DataFrame(columns = df_sample.columns)  
df_submission['planet_id'] = test_index  
df_submission.iloc[:, 1:284] = means  
df_submission.iloc[:, 284:] = stds   

# Replace inf and NaN values
df_submission = df_submission.replace([np.inf, -np.inf], np.nan)
df_submission = df_submission.fillna(0)

# Verify
print(df_submission.shape) 
df_submission.head()

I0000 00:00:1759041984.989600    1177 service.cc:148] XLA service 0x799a8c048290 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759041984.990927    1177 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1759041984.990954    1177 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1759041985.208586    1177 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


I0000 00:00:1759041987.522653    1177 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
/tmp/ipykernel_19/2261538292.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_submission = df_submission.replace([np.inf, -np.inf], np.nan)


(1, 567)


,planet_id,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,wl_10,wl_11,wl_12,wl_13,wl_14,wl_15,wl_16,wl_17,wl_18,wl_19,wl_20,wl_21,wl_22,wl_23,wl_24,wl_25,wl_26,wl_27,wl_28,wl_29,wl_30,wl_31,wl_32,wl_33,wl_34,wl_35,wl_36,wl_37,wl_38,wl_39,wl_40,wl_41,wl_42,wl_43,wl_44,wl_45,wl_46,wl_47,wl_48,wl_49,wl_50,wl_51,wl_52,wl_53,wl_54,wl_55,wl_56,wl_57,wl_58,wl_59,wl_60,wl_61,wl_62,wl_63,wl_64,wl_65,wl_66,wl_67,wl_68,wl_69,wl_70,wl_71,wl_72,wl_73,wl_74,wl_75,wl_76,wl_77,wl_78,wl_79,wl_80,wl_81,wl_82,wl_83,wl_84,wl_85,wl_86,wl_87,wl_88,wl_89,wl_90,wl_91,wl_92,wl_93,wl_94,wl_95,wl_96,wl_97,wl_98,wl_99,wl_100,wl_101,wl_102,wl_103,wl_104,wl_105,wl_106,wl_107,wl_108,wl_109,wl_110,wl_111,wl_112,wl_113,wl_114,wl_115,wl_116,wl_117,wl_118,wl_119,wl_120,wl_121,wl_122,wl_123,wl_124,wl_125,wl_126,wl_127,wl_128,wl_129,wl_130,wl_131,wl_132,wl_133,wl_134,wl_135,wl_136,wl_137,wl_138,wl_139,wl_140,wl_141,wl_142,wl_143,wl_144,wl_145,wl_146,wl_147,wl_148,wl_149,wl_150,wl_151,wl_152,wl_153,wl_154,wl_155,wl_156,wl_157,wl_158,wl_159,wl_160,wl_161,wl_162,wl_163,wl_164,wl_165,wl_166,wl_167,wl_168,wl_169,wl_170,wl_171,wl_172,wl_173,wl_174,wl_175,wl_176,wl_177,wl_178,wl_179,wl_180,wl_181,wl_182,wl_183,wl_184,wl_185,wl_186,wl_187,wl_188,wl_189,wl_190,wl_191,wl_192,wl_193,wl_194,wl_195,wl_196,wl_197,wl_198,wl_199,wl_200,wl_201,wl_202,wl_203,wl_204,wl_205,wl_206,wl_207,wl_208,wl_209,wl_210,wl_211,wl_212,wl_213,wl_214,wl_215,wl_216,wl_217,wl_218,wl_219,wl_220,wl_221,wl_222,wl_223,wl_224,wl_225,wl_226,wl_227,wl_228,wl_229,wl_230,wl_231,wl_232,wl_233,wl_234,wl_235,wl_236,wl_237,wl_238,wl_239,wl_240,wl_241,wl_242,wl_243,wl_244,wl_245,wl_246,wl_247,wl_248,wl_249,wl_250,wl_251,wl_252,wl_253,wl_254,wl_255,wl_256,wl_257,wl_258,wl_259,wl_260,wl_261,wl_262,wl_263,wl_264,wl_265,wl_266,wl_267,wl_268,wl_269,wl_270,wl_271,wl_272,wl_273,wl_274,wl_275,wl_276,wl_277,wl_278,wl_279,wl_280,wl_281,wl_282,wl_283,sigma_1,sigma_2,sigma_3,sigma_4,sigma_5,sigma_6,sigma_7,sigma_8,sigma_9,sigma_10,sigma_11,sigma_12,sigma_13,sigma_14,sigma_15,sigma_16,sigma_17,sigma_18,sigma_19,sigma_20,sigma_21,sigma_22,sigma_23,sigma_24,sigma_25,sigma_26,sigma_27,sigma_28,sigma_29,sigma_30,sigma_31,sigma_32,sigma_33,sigma_34,sigma_35,sigma_36,sigma_37,sigma_38,sigma_39,sigma_40,sigma_41,sigma_42,sigma_43,sigma_44,sigma_45,sigma_46,sigma_47,sigma_48,sigma_49,sigma_50,sigma_51,sigma_52,sigma_53,sigma_54,sigma_55,sigma_56,sigma_57,sigma_58,sigma_59,sigma_60,sigma_61,sigma_62,sigma_63,sigma_64,sigma_65,sigma_66,sigma_67,sigma_68,sigma_69,sigma_70,sigma_71,sigma_72,sigma_73,sigma_74,sigma_75,sigma_76,sigma_77,sigma_78,sigma_79,sigma_80,sigma_81,sigma_82,sigma_83,sigma_84,sigma_85,sigma_86,sigma_87,sigma_88,sigma_89,sigma_90,sigma_91,sigma_92,sigma_93,sigma_94,sigma_95,sigma_96,sigma_97,sigma_98,sigma_99,sigma_100,sigma_101,sigma_102,sigma_103,sigma_104,sigma_105,sigma_106,sigma_107,sigma_108,sigma_109,sigma_110,sigma_111,sigma_112,sigma_113,sigma_114,sigma_115,sigma_116,sigma_117,sigma_118,sigma_119,sigma_120,sigma_121,sigma_122,sigma_123,sigma_124,sigma_125,sigma_126,sigma_127,sigma_128,sigma_129,sigma_130,sigma_131,sigma_132,sigma_133,sigma_134,sigma_135,sigma_136,sigma_137,sigma_138,sigma_139,sigma_140,sigma_141,sigma_142,sigma_143,sigma_144,sigma_145,sigma_146,sigma_147,sigma_148,sigma_149,sigma_150,sigma_151,sigma_152,sigma_153,sigma_154,sigma_155,sigma_156,sigma_157,sigma_158,sigma_159,sigma_160,sigma_161,sigma_162,sigma_163,sigma_164,sigma_165,sigma_166,sigma_167,sigma_168,sigma_169,sigma_170,sigma_171,sigma_172,sigma_173,sigma_174,sigma_175,sigma_176,sigma_177,sigma_178,sigma_179,sigma_180,sigma_181,sigma_182,sigma_183,sigma_184,sigma_185,sigma_186,sigma_187,sigma_188,sigma_189,sigma_190,sigma_191,sigma_192,sigma_193,sigma_194,sigma_195,sigma_196,sigma_197,sigma_198,sigma_199,sigma_200,sigma_201,sigma_202,sigma_203,sigma_204,sigma_205,sigma_206,sigma_207,sigma_208,sigma_209,sigma_210,sigma_211,sigma_212,sigma_213,sigma_214,sigma_215,sigma_216,sigma_217,sigma_218,sigma_219,sigma_220,sigma_221,sigma_222,sigm

In [21]:
df_submission.to_csv('/kaggle/working/submission.csv', index=False)